In [1]:
from scipy.stats import multivariate_normal as mvn
import numpy as np
import random
from numpy import linalg as LA
import sklearn as sklearn
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from pylab import *
from sklearn.cluster import KMeans
from numpy.core.umath_tests import matrix_multiply as mm

# %matplotlib qt

C:\Users\Sayan\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  # Remove the CWD from sys.path while we load stuff.


In [2]:
# def em_gmm_orig(xs, pis, mus, sigmas, tol=0.000001, max_iter=1000):

#     n, p = xs.shape
#     k = len(pis)
#     iter = 0
#     ll_old = 0
#     for i in range(max_iter):
#         exp_A = []
#         exp_B = []
#         ll_new = 0

#         # E-step
#         ws = np.zeros((k, n))
#         for j in range(len(mus)):
#             for i in range(n):
#                 ws[j, i] = pis[j] * mvn(mus[j], sigmas[j]).pdf(xs[i])
#         ws /= ws.sum(0)

#         # M-step
#         pis = np.zeros(k)
#         for j in range(len(mus)):
#             for i in range(n):
#                 pis[j] += ws[j, i]
#         pis /= n

#         mus = np.zeros((k, p))
#         for j in range(k):
#             for i in range(n):
#                 mus[j] += ws[j, i] * xs[i]
#             mus[j] /= ws[j, :].sum()

#         sigmas = np.zeros((k, p, p))
#         for j in range(k):
#             for i in range(n):
#                 ys = np.reshape(xs[i]- mus[j], (2,1))
#                 sigmas[j] += ws[j, i] * np.dot(ys, ys.T)
#             sigmas[j] /= ws[j,:].sum()

#         # update complete log likelihoood
#         ll_new = 0.0
#         for i in range(n):
#             s = 0
#             for j in range(k):
#                 s += pis[j] * mvn(mus[j], sigmas[j]).pdf(xs[i])
#             ll_new += np.log(s)

#         if np.abs(ll_new - ll_old) < tol:
#             break
        
        
#         print('iter =', iter, ', error =',np.abs(ll_new - ll_old))
#         iter +=1
#         ll_old = ll_new

#     return ll_new, pis, mus, sigmas

In [15]:
def em_gmm_vect(xs, pis, mus, sigmas, tol=0.000001, max_iter=1):

    n, p = xs.shape
    k = len(pis)
    iter = 0

    ll_old = 0
    for i in range(max_iter):
        exp_A = []
        exp_B = []
        ll_new = 0

        # E-step
        ws = np.zeros((k, n))
        for j in range(k):
            ws[j, :] = pis[j] * mvn(mus[j], sigmas[j]).pdf(xs)
        ws /= ws.sum(0)

        # M-step
        pis = ws.sum(axis=1)
        pis /= n

        mus = np.dot(ws, xs)
        mus /= ws.sum(1)[:, None]

        sigmas = np.zeros((k, p, p))
        for j in range(k):
            ys = xs - mus[j, :]
            print('--------------------------------------------------------')
#             print( (ws[j,:,None,None] * mm(ys[:,:,None], ys[:,None,:])) )
#             print( (ws[j,:,None,None]).shape )
            print( ((ys[:,:,None])))
            print( ((ys[:,None,:])))
            print('\n\n')
#             print( (ws[j,:,None,None] * mm(ys[:,:,None], ys[:,None,:])).sum(axis=0) )
            print('--------------------------------------------------------')

            sigmas[j] = (ws[j,:,None,None] * mm(ys[:,:,None], ys[:,None,:])).sum(axis=0)
        sigmas /= ws.sum(axis=1)[:,None,None]

        # update complete log likelihoood
        ll_new = 0.0
        for i in range(n):
            s = 0
            for j in range(k):
                s += pis[j] * mvn(mus[j], sigmas[j]).pdf(xs[i])
            ll_new += np.log(s)

        if np.abs(ll_new - ll_old) < tol:
            break

        print('iter =', iter, ', error =',np.abs(ll_new - ll_old))
        iter +=1
        ll_old = ll_new

    return ll_new, pis, mus, sigmas

In [4]:
# def em_gmm_vect(xs, pis, mus, sigmas, tol=0.000001, max_iter=1000):

#     n, p = xs.shape
#     k = len(pis)
#     iter = 0

#     ll_old = 0
#     for i in range(max_iter):
#         exp_A = []
#         exp_B = []
#         ll_new = 0

#         # E-step
#         ws = np.zeros((k, n))
#         for j in range(k):
#             ws[j, :] = pis[j] * mvn(mus[j], sigmas[j]).pdf(xs)
#         ws /= ws.sum(0)

#         # M-step
#         pis = ws.sum(axis=1)
#         pis /= n

#         mus = np.dot(ws, xs)
#         mus /= ws.sum(1)[:, None]

#         sigmas = np.zeros((k, p, p))
#         for j in range(k):
#             ys = xs - mus[j, :]
#             sigmas[j] = (ws[j,:,None,None] * mm(ys[:,:,None], ys[:,None,:])).sum(axis=0)
#         sigmas /= ws.sum(axis=1)[:,None,None]

#         # update complete log likelihoood
#         ll_new = 0
#         for pi, mu, sigma in zip(pis, mus, sigmas):
#             ll_new += pi*mvn(mu, sigma).pdf(xs)
#         ll_new = np.log(ll_new).sum()

#         if np.abs(ll_new - ll_old) < tol:
#             break

#         print('iter =', iter, ', error =',np.abs(ll_new - ll_old))
#         iter +=1
#         ll_old = ll_new

#     return ll_new, pis, mus, sigmas

In [5]:
# def em_gmm_eins(xs, pis, mus, sigmas, tol=0.0000001, max_iter=1000):

#     n, p = xs.shape
#     k = len(pis)
#     iter = 0
    
#     ll_old = 0
#     for i in range(max_iter):
#         exp_A = []
#         exp_B = []
#         ll_new = 0

#         # E-step
#         ws = np.zeros((k, n))
#         for j, (pi, mu, sigma) in enumerate(zip(pis, mus, sigmas)):
#             ws[j, :] = pi * mvn(mu, sigma).pdf(xs)
#         ws /= ws.sum(0)

#         # M-step
#         pis = np.einsum('kn->k', ws)/n
#         mus = np.einsum('kn,np -> kp', ws, xs)/ws.sum(1)[:, None]
#         sigmas = np.einsum('kn,knp,knq -> kpq', ws,
#             xs-mus[:,None,:], xs-mus[:,None,:])/ws.sum(axis=1)[:,None,None]

#         # update complete log likelihoood
#         ll_new = 0
#         for pi, mu, sigma in zip(pis, mus, sigmas):
#             ll_new += pi*mvn(mu, sigma).pdf(xs)
#         ll_new = np.log(ll_new).sum()

#         if np.abs(ll_new - ll_old) < tol:
#             break
        
#         print('iter =', iter, ', error =',np.abs(ll_new - ll_old))
#         iter += 1
#         ll_old = ll_new

#     return ll_new, pis, mus, sigmas

In [6]:
def generate_data(generators, N, pi):
#     X_data = np.concatenate([np.random.multivariate_normal(mu, sigma, int(pi*N))
#                     for pi, mu, sigma in zip(pii, mean, cov)])
    draw = np.random.choice([0, 1, 2], N, p=pi.tolist())
    unique, counts = np.unique(draw, return_counts=True)
    
    mixture_info = (dict(zip(unique, counts)))
    
    X_Data = [np.array(generators[i].rvs()) for i in draw]
    X_Data = np.array(X_Data)
    return X_Data.T, mixture_info

def ploting(X_Data):

    if X_Data.shape[1] == 2:
        fig, ax = plt.subplots(num=1, clear=True)
        handle = ax.plot(X_Data[:,0], X_Data[:,1], '.')
        plt.axis('equal')
#         plt.show()
        
    elif X_Data.shape[1] == 3:
        fig = plt.figure()
        ax = plt.axes(projection='3d')
        ax.plot3D(X_Data[:,0], X_Data[:,1], X_Data[:,2], '.')
        
    else:
        print("Can't plot dimension > 3 !!!")

    return None

In [16]:
pi = np.array([0.4, 0.3, 0.3])
generators = [
    mvn([-10, 10], [[5, 0], [0, 7]]),             
    mvn([2, -5], [[10, 0], [0, 3]]), 
    mvn([1, 0], [[5, 0], [0, 15]])]
N = 10
K = 3
xs, mixture_info =  generate_data(generators, N, pi)

# initial guesses for parameters
pis = np.random.random(3)
pis /= pis.sum()
mus = np.random.random((3,2))
sigmas = np.array([np.eye(2)] * 3)


ll1, pis1, mus1, sigmas1 = em_gmm_vect(xs.T, pi, mus, sigmas)
print(pis1)
print(mus1)
print(sigmas1)

--------------------------------------------------------
[[[  0.56440998]
  [ -7.24423831]]

 [[ -7.65212477]
  [  7.78679354]]

 [[  1.96063783]
  [ -4.15599534]]

 [[  2.52227244]
  [ -1.23387835]]

 [[  1.00059322]
  [  1.11465858]]

 [[-12.19538448]
  [ 10.39996863]]

 [[  2.26104999]
  [ -1.48455458]]

 [[  4.4395305 ]
  [ -6.15280053]]

 [[ -0.888508  ]
  [  2.30595825]]

 [[ -7.39991002]
  [ 11.55585744]]]
[[[  0.56440998  -7.24423831]]

 [[ -7.65212477   7.78679354]]

 [[  1.96063783  -4.15599534]]

 [[  2.52227244  -1.23387835]]

 [[  1.00059322   1.11465858]]

 [[-12.19538448  10.39996863]]

 [[  2.26104999  -1.48455458]]

 [[  4.4395305   -6.15280053]]

 [[ -0.888508     2.30595825]]

 [[ -7.39991002  11.55585744]]]



--------------------------------------------------------
--------------------------------------------------------
[[[ -1.63034925]
  [ -3.44847159]]

 [[ -9.846884  ]
  [ 11.58256026]]

 [[ -0.2341214 ]
  [ -0.36022862]]

 [[  0.32751321]
  [  2.56188837]]

 [